In [13]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.launching.run_super import *
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Node,
    Experiment,
    save_graphs,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
directory = directory_project
# update_plate_info(directory)

In [2]:
directory = "/projects/0/einf914/agg/"
update_plate_info(directory)

analysed:   0%|          | 0/441 [00:00<?, ?it/s]

In [9]:
all_folders = get_current_folders(directory)

In [10]:
folders = all_folders.loc[all_folders["Plate"] == "737"]
print(len(folders))

220


In [11]:
folders = folders.sort_values(by=["datetime"], ignore_index=True)

***Create graphs***

In [20]:
num_parallel = 128
time = "5:00:00"
args = [directory]
run_parallel(
    "extract_nx_graph.py",
    args,
    folders,
    num_parallel,
    time,
    "extract_nx",
    cpus=128,
    node="fat",
)

Submitted batch job 507919
Submitted batch job 507921


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [69]:
plates = [22, 10, 76, 26, 152, 102]

folders = all_folders.loc[
    (all_folders["/Analysis/nx_graph_pruned.p"]) & (all_folders["Plate"].isin(plates))
]
len(folders)

336

***Extract Width***

In [6]:
num_parallel = 128
time = "24:00:00"
skip = False
resolution = 50
args = [directory, skip, resolution]
run_parallel(
    "extract_width.py",
    args,
    folders,
    num_parallel,
    time,
    "extract_width2",
    cpus=32,
    node="fat",
)

Submitted batch job 508441
Submitted batch job 508442


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 508443
Submitted batch job 508444
Submitted batch job 508445


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [41]:
plates = [22, 10, 76, 26, 152, 102]

folders = all_folders.loc[
    (1 - all_folders["/Analysis/nx_graph_pruned_width.p"])
    & (all_folders["Plate"].isin(plates))
    & (all_folders["/Analysis/nx_graph_pruned.p"])
]
# folders.loc[folders['folder'] == '20210110_1523_Plate30']
# int(list(folders['folder'])[0].split('_')[-1][5:])
len(folders)

0

In [4]:
plates = [22, 10, 76, 26, 152, 102]

folders = all_folders.loc[
    (all_folders["/Analysis/nx_graph_pruned_width.p"])
    & (all_folders["Plate"].isin(plates))
]
# folders.loc[folders['folder'] == '20210110_1523_Plate30']
# int(list(folders['folder'])[0].split('_')[-1][5:])
len(folders)

336

***Identify Nodes***

In [7]:
num_parallel = 32
time = "1:00:00"
limit = 55
skip = False
args = [directory, limit, skip]
run_parallel_all_time(
    "extract_nodes.py",
    args,
    folders,
    num_parallel,
    time,
    "node_id",
    cpus=32,
    node="fat",
)

Submitted batch job 535639


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [14]:
plates = list(set(list(folders["unique_id"].values)))
args = [directory]
num_parallel = 128

for unique_id in plates:
    select = folders.loc[folders["unique_id"] == unique_id]
    time = "2:00:00"
    run_parallel(
        "track_nodes.py",
        args,
        select,
        num_parallel,
        time,
        "track_node",
        cpus=128,
        node="fat",
        name_job="track.sh",
    )

time = "30:00"
run_parallel_all_time(
    "make_labeled_graphs.py",
    args,
    folders,
    num_parallel,
    time,
    "make_graphs",
    cpus=128,
    node="fat",
    dependency=True,
    name_job="track.sh",
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1089373


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1089374
Submitted batch job 1089375


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [10]:
folders = folders.sort_values("datetime")
select = folders.iloc[:10]
exp = Experiment(directory)
exp.load(select, suffix="_labeled")

2021-09-01 21:39:00
1


/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/pipeline/functions/image_processing/experiment_class_surf.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.folders["date"], format="%d.%m.%Y, %H:%M:"


FileNotFoundError: [Errno 2] No such file or directory: '/projects/0/einf914/agg/20210901_2139_Plate34/Analysis/nx_graph_pruned_labeled.p'

***Hyphae extraction***

In [6]:
num_parallel = 32
time = "8:00:00"
limit = 55
version = 11
labeled = True
args = [directory, limit, version, labeled]
run_parallel_all_time(
    "hyphae_extraction.py",
    args,
    folders,
    num_parallel,
    time,
    "hyphae",
    cpus=32,
    node="fat",
)

Submitted batch job 551041


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
